# Traversing the World of Warcraft Travel Graph

This goal of this notebook is to showcase how to craft and traverse knowledge graphs using graph database technologies. In this example, we'll leverage information from a popular MMORPG ([World of Warcraft](https://worldofwarcraft.com/)) to create a dataset, load it into a graph database ([Amazon Neptune](https://aws.amazon.com/neptune/)), and traverse the graph using [Apache Tinkerpop Gremlin](https://tinkerpop.apache.org/). By the end, we'll be able to craft simple Gremlin queries that help to answer answer difficult questions about our highly interrelated graph data.

![Full Map of Azeroth Travel Routes](../maps/azeroth.png "Azeroth")
Source: [Classic WoW: The One Map to Rule Them All](https://www.reddit.com/r/classicwow/comments/dcu95i/one_map_to_rule_them_all_one_map_to_find_them_one/), a modified version of maps found in the [BradyGames Official Strategy Guide](https://www.amazon.com/World-Warcraft-Official-Strategy-Bradygames/dp/0744004055).

## 0. Background

Much like on Earth, in World of Warcraft, there are various forms of transportation. Each form of transportation has different unique properties, from speed, cost, prerequisites, and more. At a glance:
- **Flights**: Paid, on-demand, automated personal flights between discrete start and end flightmasters. Requires a nondisjoint path of discovered, connected flight masters available to the player's faction. Gryphons for Alliance, Batriders for Horde.
- **Boats**: Free, cyclical, shared, self-guided, faction agnostic cross-continent travel, located near/on the edge of certain cities. Some are based in neutral cities while others are based near alliance cities.
- **Tram**: Free, cyclical, shared, self-guided, high speed railway located inside certain Alliance cities, with no faction requirement for riding.
- **Zeppelins**: Free, cyclical, shared, self-guided high speed long distance airborne ships, located outside certain Horde cities.
- **Walking/Swimming**: Free, on-demand, self-guided travel. Can be modified by various factors like mount (+60% regular or +100% epic ground mount, speed potion, Sprint (Rogue ability), Blink (Mage ability)). Your mileage may vary (literally)! Abilities and mounts all have firm level and gold unlock requirements.

Other interesting forms of transit:
- **Mage Portal**: Reagent costing Mage-only spell, offering instantaneous transit for the caster and party/raid members from any origin (where spell is cast) to one of the same-faction major cities.
- **Ritual of Summoning**: Reagent costing Warlock-only spell, offering instantaneous transit for the target player to the location of the caster. Requires a Warlock (learned at level 42) to cast, and two party members to channel.
- **Summoning Stone**: Free, instaneous transit, bringing the target player to the summoning stone location. Located outside dungeons/instances. Requires three party members of any level/class to channel.

#### In short,

When it comes down to it, players have choice as to how they choose to navigate the world around them. In such a large world, time becomes one of the biggest factors players take into account when choosing a transportation methodology. However, many of the fastest transportation methods are gated behind gold or level requirements, and even have danger considerations (if using transport in/near enemy factions). Examples of the many methods from a subset of the Eastern Kingdoms graph:

![Zoomed in view of Eastern Kingdoms Routes](../maps/ek-closeup.png "Eastern Kingdoms Closeup")
Source: [Classic WoW: The One Map to Rule Them All](https://www.reddit.com/r/classicwow/comments/dcu95i/one_map_to_rule_them_all_one_map_to_find_them_one/)


Regardless of specific implementation details in World of Warcraft, there are lots of questions that geospatial knowledge graphs help to answer.


#### For the players...

Some simple examples include: 
   - What is the quickest path (**time**) between two points?
   - What is the shortest path (**distance**) between two points?
   - What is the safest path (**least contested territory**) between two points?
   - What are the longest (**time**) single leg flights in the game?

While some more complex questions also start to surface:
   - What is the quickest way to travel between two points without venturing to neutral territory **(time*safety)**?
   - What are the longest (**time**) multipart flights in the game?
   - What are the most common (**cardinality**) connecting flight points in the game?
   
  
#### For game developers...

Questions about hypothetical game states or prospective changes to the game can easily be answered with graph traversals. Questions in this realm include:
   - If a flight master (vertex) is removed or able to be killed:
       - **Which destinations does this prevent** travel to?
       - **How does this effect travel time** for routes that passed through here?
   - If a flight's speed or cost is altered:
       - How does this effect **overall gold sink** (increase? decrease?)
       - Do other forms/combiantions of transportation become more ideal routes from a speed or cost perspective?


While it is possible to perform this type of analysis with a traditional relational or key-value NoSQL database, graph databases like Amazon Neptune are purpose-built to improve the experience of crafting resilient queries, with an intuitive yet robust underlying data structure, making it significantly easier to answer questions that involve interrelated data.

## 1. Create the dataset

To keep things simple, we'll craft the first version of our dataset exclusively with flight points. While the quasi-schemaless nature of graph databases means that it will be easy to add different types of transportation methods later, we'll want to be mindful of standardizing naming conventions for properties that we expect to be consistent across our different methodologies. 

**In graphs, there are two primary top level entities:**
- Vertex (A node, an object or entity)
- Edge (Denotes a relationship between two objects)

**When thinking about our entire World of Warcraft travel graph, this takes the following generalizable form:**
- Vertex: Location
- Edge: Path travellable between two locations

**In this case, for flight masters:**
- Vertex: Flightmaster (or a city, as each city has one* flight master)
- Edge: Flight route between individual flightmasters

### To properly represent flight point locations, we'll look at the following parameters

With each flightpoint representing a `vertex` consisting of a number of properties:

- `vertex[0]` id (str), unique identifier for the flight point
- `vertex[1]` shortcode (str), easily understandable abbreviation (similar to an airport code)
- `vertex[2]` name (str), full city name
- `vertex[3]` zone (str), broader zone that the city exists within
- `vertex[4]` continent (str), broader continent that the city and zone reside within
- `vertex[5]` faction (str), does this city's flight master have a particular faction?
- `vertex[6]` discovered (bool), has the city been discovered? Note: this may 
- `vertex[7]` label (str), designate vertex type. Only one type when considering flightpoints, but will be a valuable property when we add other edge types later.


There are various ways to import data into Amazon Neptune -- for this example, we'll be using Apache Tinkerpop Gremlin directly. While in the notebook we'll be showcasing ingesting the data into this Jupyter Notebook and then performing vertex adds to our Neptune graph, for a larger dataset that may not be able to fit into memory in a Jupyter notebook like this, it would make sense to use [Amazon S3 to bulk load data in parallel](https://docs.aws.amazon.com/neptune/latest/userguide/bulk-load-data.html) directly to Neptune.

Both the flightmaster vertex file [`vertices.csv`](./data/vertices.csv) and flight path edges file [`edges.csv`](`./data/edges.csv`) included in the directory are in valid Gremlin format for bulk upload.

### Edges will require slightly different properties,

Given that edges represent the relationships between vertices, the edges in this case will represent the flight path segments players can take between flight masters. Codifying faction-specificity is a necessity, with concepts like additional prerequisites (discoverd, has sufficient gold to pay fare) and duration being valuable additions for answering questions.  

- `edge[0]` id (str), unique identifier for the flight path
- `edge[1]` from (str), segment origin ID
- `edge[2]` to (str), segment destination ID
- `edge[3]` label (str), type of transportation ("flightpath")
- `edge[4]` duration (single), what is the duration (in seconds) for this route?
- `edge[5]` faction (str), is this path available to Horde, Alliance, or both?
- `edge[6]` prereqs (str), does this path require any prerequisites to qualify for?


#### Housekeeping extras: remove all edges and vertices from graph

In [139]:
%%gremlin
g.V().drop()

Console 
 
 

 
 
 
 
 
 
 

 
 
 
 
 
 Page Size: 
 
 Total Results: 0

In [132]:
%%gremlin
g.E().drop()

Console 
 
 

 
 
 
 
 
 
 

 
 
 
 
 
 Page Size: 
 
 Total Results: 0

## 2. Load the data

### Ingest the data into the Jupyter Notebook

Import our vertices (flight locations) and edges (flightpaths) from our data sources, `vertices.csv` and `edges.csv`

In [140]:
import csv

#import vertices
with open('../data/vertices.csv', newline='') as csvfile:
    vertices = list(csv.reader(csvfile))

print(vertices)

[['~id', ' shortcode:String', ' name:String', ' zone:String', ' continent:String', ' faction:String', ' discovered:Bool', ' ~label'], ['1', ' "LH"', ' "Light\'s Hope Chapel"', ' "Eastern Plaguelands"', ' "Eastern Kingdoms"', ' "Alliance"', ' True', ' flightpoint'], ['2', ' "CW"', ' "Chillwind Camp"', ' "Western Plaguelands"', ' "Eastern Kingdoms"', ' "Alliance"', ' True', ' flightpoint'], ['3', ' "AP"', ' "Aerie Peak"', ' "The Hinterlands"', ' "Eastern Kingdoms"', ' "Alliance"', ' True', ' flightpoint'], ['4', ' "SS"', ' "Southshore"', ' "Hillsbrad Foothills"', ' "Eastern Kingdoms"', ' "Alliance"', ' True', ' flightpoint'], ['5', ' "RP"', ' "Refuge Pointe"', ' "Arathi Highlands"', ' "Eastern Kingdoms"', ' "Alliance"', ' True', ' flightpoint'], ['6', ' "MH"', ' "Menethil Harbor"', ' "Wetlands"', ' "Eastern Kingdoms"', ' "Alliance"', ' True', ' flightpoint'], ['7', ' "IF"', ' "Ironforge"', ' "Dun Morogh"', ' "Eastern Kingdoms"', ' "Alliance"', ' True', ' flightpoint'], ['8', ' "TH"', ' "

In [136]:
# import edges
with open('../data/edges.csv', newline='') as csvfile:
    edges = list(csv.reader(csvfile))

print(edges)

[['~id', '~from', '~to', '~label', 'duration:Double', 'faction:String', 'prereqs:String'], ['LH-CW', '1', '2', 'flightpath', '', 'Alliance', 'discovered'], ['LH-AP', '1', '3', 'flightpath', '', 'Alliance', 'discovered'], ['LH-IF', '1', '7', 'flightpath', '', 'Alliance', 'discovered'], ['CW-AP', '2', '3', 'flightpath', '', 'Alliance', 'discovered'], ['CW-SS', '2', '4', 'flightpath', '', 'Alliance', 'discovered'], ['CW-IF', '2', '7', 'flightpath', '', 'Alliance', 'discovered'], ['AP-SS', '3', '4', 'flightpath', '', 'Alliance', 'discovered'], ['AP-RP', '3', '5', 'flightpath', '', 'Alliance', 'discovered'], ['SS-RP', '4', '5', 'flightpath', '', 'Alliance', 'discovered'], ['SS-MH', '4', '6', 'flightpath', '', 'Alliance', 'discovered'], ['SS-IF', '4', '7', 'flightpath', '', 'Alliance', 'discovered'], ['RP-MH', '5', '6', 'flightpath', '', 'Alliance', 'discovered'], ['RP-TH', '5', '8', 'flightpath', '', 'Alliance', 'discovered'], ['MH-TH', '6', '8', 'flightpath', '', 'Alliance', 'discovered'],

### Iterate across the array of vertices to craft a list of gremlin "add vertex" operations

In [141]:
# Generate Gremlin queries for adding vertices
for location in vertices[1:]:
    vertex_add_query = 'g.addV("flightpath").property(id,\"'+location[0]+'\").property("shortcode", '+location[1]+').property("name", '+location[2]+').property("zone", '+location[3]+').property("continent", '+location[4]+').property("faction", '+location[5]+').property("discovered", "True").next()'
    print(vertex_add_query)
    #vertex_adds.append('g.addV('+location[7]+').property(id,'+location[0]+').property(shortcode, '+location[1]+').property(name, '+location[2]+').property(zone, '+location[3]+').property(continent, '+location[4]+').property(faction, '+location[5]+').property(discovered, '+location[6]+')')    

g.addV("flightpath").property(id,"1").property("shortcode",  "LH").property("name",  "Light's Hope Chapel").property("zone",  "Eastern Plaguelands").property("continent",  "Eastern Kingdoms").property("faction",  "Alliance").property("discovered", "True").next()
g.addV("flightpath").property(id,"2").property("shortcode",  "CW").property("name",  "Chillwind Camp").property("zone",  "Western Plaguelands").property("continent",  "Eastern Kingdoms").property("faction",  "Alliance").property("discovered", "True").next()
g.addV("flightpath").property(id,"3").property("shortcode",  "AP").property("name",  "Aerie Peak").property("zone",  "The Hinterlands").property("continent",  "Eastern Kingdoms").property("faction",  "Alliance").property("discovered", "True").next()
g.addV("flightpath").property(id,"4").property("shortcode",  "SS").property("name",  "Southshore").property("zone",  "Hillsbrad Foothills").property("continent",  "Eastern Kingdoms").property("faction",  "Alliance").property("dis

### Execute the above `g.addV()` commands as gremlin queries:
(copy and paste the output from the above cell)

In [142]:
%%gremlin
g.addV("flightpath").property(id,"1").property("shortcode",  "LH").property("name",  "Light's Hope Chapel").property("zone",  "Eastern Plaguelands").property("continent",  "Eastern Kingdoms").property("faction",  "Alliance").property("discovered", "True").next()
g.addV("flightpath").property(id,"2").property("shortcode",  "CW").property("name",  "Chillwind Camp").property("zone",  "Western Plaguelands").property("continent",  "Eastern Kingdoms").property("faction",  "Alliance").property("discovered", "True").next()
g.addV("flightpath").property(id,"3").property("shortcode",  "AP").property("name",  "Aerie Peak").property("zone",  "The Hinterlands").property("continent",  "Eastern Kingdoms").property("faction",  "Alliance").property("discovered", "True").next()
g.addV("flightpath").property(id,"4").property("shortcode",  "SS").property("name",  "Southshore").property("zone",  "Hillsbrad Foothills").property("continent",  "Eastern Kingdoms").property("faction",  "Alliance").property("discovered", "True").next()
g.addV("flightpath").property(id,"5").property("shortcode",  "RP").property("name",  "Refuge Pointe").property("zone",  "Arathi Highlands").property("continent",  "Eastern Kingdoms").property("faction",  "Alliance").property("discovered", "True").next()
g.addV("flightpath").property(id,"6").property("shortcode",  "MH").property("name",  "Menethil Harbor").property("zone",  "Wetlands").property("continent",  "Eastern Kingdoms").property("faction",  "Alliance").property("discovered", "True").next()
g.addV("flightpath").property(id,"7").property("shortcode",  "IF").property("name",  "Ironforge").property("zone",  "Dun Morogh").property("continent",  "Eastern Kingdoms").property("faction",  "Alliance").property("discovered", "True").next()
g.addV("flightpath").property(id,"8").property("shortcode",  "TH").property("name",  "Thelsamar").property("zone",  "Loch Modan").property("continent",  "Eastern Kingdoms").property("faction",  "Alliance").property("discovered", "True").next()
g.addV("flightpath").property(id,"9").property("shortcode",  "TP").property("name",  "Thorium Point").property("zone",  "Searing Gorge").property("continent",  "Eastern Kingdoms").property("faction",  "Alliance").property("discovered", "True").next()
g.addV("flightpath").property(id,"10").property("shortcode",  "MV").property("name",  "Morgan's Vigil").property("zone",  "Burning Steppes").property("continent",  "Eastern Kingdoms").property("faction",  "Alliance").property("discovered", "True").next()
g.addV("flightpath").property(id,"11").property("shortcode",  "SW").property("name",  "Stormwind City").property("zone",  "Elwynn Forest").property("continent",  "Eastern Kingdoms").property("faction",  "Alliance").property("discovered", "True").next()
g.addV("flightpath").property(id,"12").property("shortcode",  "LS").property("name",  "Lakeshire").property("zone",  "Redridge Mountains").property("continent",  "Eastern Kingdoms").property("faction",  "Alliance").property("discovered", "True").next()
g.addV("flightpath").property(id,"13").property("shortcode",  "SH").property("name",  "Sentinel Hill").property("zone",  "Westfall").property("continent",  "Eastern Kingdoms").property("faction",  "Alliance").property("discovered", "True").next()
g.addV("flightpath").property(id,"14").property("shortcode",  "DW").property("name",  "Darkshire").property("zone",  "Duskwood").property("continent",  "Eastern Kingdoms").property("faction",  "Alliance").property("discovered", "True").next()
g.addV("flightpath").property(id,"15").property("shortcode",  "NK").property("name",  "Nethergarde Keep").property("zone",  "Blasted Lands").property("continent",  "Eastern Kingdoms").property("faction",  "Alliance").property("discovered", "True").next()
g.addV("flightpath").property(id,"16").property("shortcode",  "BB").property("name",  "Booty Bay").property("zone",  "Stranglethorn Vale").property("continent",  "Eastern Kingdoms").property("faction",  "Neutral").property("discovered", "True").next()
g.addV("flightpath").property(id,"17").property("shortcode",  "UC").property("name",  "The Undercity").property("zone",  "Tirisfal Glades").property("continent",  "Eastern Kingdoms").property("faction",  "Horde").property("discovered", "True").next()
g.addV("flightpath").property(id,"18").property("shortcode",  "SP").property("name",  "The Sepulcher").property("zone",  "Silverpine Forest").property("continent",  "Eastern Kingdoms").property("faction",  "Horde").property("discovered", "True").next()
g.addV("flightpath").property(id,"19").property("shortcode",  "TM").property("name",  "Tarren Mill").property("zone",  "Hillsbrad Foothills").property("continent",  "Eastern Kingdoms").property("faction",  "Horde").property("discovered", "True").next()
g.addV("flightpath").property(id,"20").property("shortcode",  "RV").property("name",  "Raventusk Village").property("zone",  "The Hinterlands").property("continent",  "Eastern Kingdoms").property("faction",  "Horde").property("discovered", "True").next()
g.addV("flightpath").property(id,"21").property("shortcode",  "HF").property("name",  "Hammerfall").property("zone",  "Arathi Highlands").property("continent",  "Eastern Kingdoms").property("faction",  "Horde").property("discovered", "True").next()
g.addV("flightpath").property(id,"22").property("shortcode",  "KG").property("name",  "Kargath").property("zone",  "Badlands").property("continent",  "Eastern Kingdoms").property("faction",  "Horde").property("discovered", "True").next()
g.addV("flightpath").property(id,"23").property("shortcode",  "FC").property("name",  "Flame Crest").property("zone",  "Burning Steppes").property("continent",  "Eastern Kingdoms").property("faction",  "Horde").property("discovered", "True").next()
g.addV("flightpath").property(id,"24").property("shortcode",  "ST").property("name",  "Stonard").property("zone",  "Swamp of Sorrows").property("continent",  "Eastern Kingdoms").property("faction",  "Horde").property("discovered", "True").next()
g.addV("flightpath").property(id,"25").property("shortcode",  "GG").property("name",  "Grom'gol").property("zone",  "Stranglethorn Vale").property("continent",  "Eastern Kingdoms").property("faction",  "Horde").property("discovered", "True").next()
g.addV("flightpath").property(id,"26").property("shortcode",  "QD").property("name",  "Quel'danas").property("zone",  "Silvermoon City").property("continent",  "Eastern Kingdoms").property("faction",  "Horde").property("discovered", "True").next()
g.addV("flightpath").property(id,"27").property("shortcode",  "DN").property("name",  "Rut'theran Village").property("zone",  "Darnassus").property("continent",  "Kalimdor").property("faction",  "Alliance").property("discovered", "True").next()
g.addV("flightpath").property(id,"28").property("shortcode",  "AU").property("name",  "Auberdine").property("zone",  "Darkshore").property("continent",  "Kalimdor").property("faction",  "Alliance").property("discovered", "True").next()
g.addV("flightpath").property(id,"29").property("shortcode",  "MG").property("name",  "Nighthaven").property("zone",  "Moonglade").property("continent",  "Kalimdor").property("faction",  "Alliance").property("discovered", "True").next()
g.addV("flightpath").property(id,"30").property("shortcode",  "TG").property("name",  "Talonbranch Glade").property("zone",  "Felwood").property("continent",  "Kalimdor").property("faction",  "Alliance").property("discovered", "True").next()
g.addV("flightpath").property(id,"31").property("shortcode",  "EL").property("name",  "Everlook").property("zone",  "Winterspring").property("continent",  "Kalimdor").property("faction",  "Alliance").property("discovered", "True").next()
g.addV("flightpath").property(id,"32").property("shortcode",  "SP").property("name",  "Stonetalon Peak").property("zone",  "Stoletalon Mountains").property("continent",  "Kalimdor").property("faction",  "Alliance").property("discovered", "True").next()
g.addV("flightpath").property(id,"33").property("shortcode",  "AS").property("name",  "Astranaar").property("zone",  "Ashenvale").property("continent",  "Kalimdor").property("faction",  "Alliance").property("discovered", "True").next()
g.addV("flightpath").property(id,"34").property("shortcode",  "NP").property("name",  "Nijel's Point").property("zone",  "Desolace").property("continent",  "Kalimdor").property("faction",  "Alliance").property("discovered", "True").next()
g.addV("flightpath").property(id,"35").property("shortcode",  "TR").property("name",  "Talrendis Point").property("zone",  "Azshara").property("continent",  "Kalimdor").property("faction",  "Alliance").property("discovered", "True").next()
g.addV("flightpath").property(id,"36").property("shortcode",  "RA").property("name",  "Ratchet").property("zone",  "The Barrens").property("continent",  "Kalimdor").property("faction",  "Alliance").property("discovered", "True").next()
g.addV("flightpath").property(id,"37").property("shortcode",  "FM").property("name",  "Feathermoon Stronghold").property("zone",  "Feralas").property("continent",  "Kalimdor").property("faction",  "Alliance").property("discovered", "True").next()
g.addV("flightpath").property(id,"38").property("shortcode",  "TN").property("name",  "Thalanaar").property("zone",  "Thousand Needles").property("continent",  "Kalimdor").property("faction",  "Alliance").property("discovered", "True").next()
g.addV("flightpath").property(id,"39").property("shortcode",  "TI").property("name",  "Theramore Isle").property("zone",  "Dustwallow Marsh").property("continent",  "Kalimdor").property("faction",  "Alliance").property("discovered", "True").next()
g.addV("flightpath").property(id,"40").property("shortcode",  "CH").property("name",  "Cenarion Hold").property("zone",  "Silithus").property("continent",  "Kalimdor").property("faction",  "Alliance").property("discovered", "True").next()
g.addV("flightpath").property(id,"41").property("shortcode",  "MR").property("name",  "Marshal's Refuge").property("zone",  "Un'goro Crater").property("continent",  "Kalimdor").property("faction",  "Alliance").property("discovered", "True").next()
g.addV("flightpath").property(id,"42").property("shortcode",  "GZ").property("name",  "Gadgetzan").property("zone",  "Tanaris").property("continent",  "Kalimdor").property("faction",  "Alliance").property("discovered", "True").next()
g.addV("flightpath").property(id,"43").property("shortcode",  "MG").property("name",  "Nighthaven").property("zone",  "Moonglade").property("continent",  "Kalimdor").property("faction",  "Horde").property("discovered", "True").next()
g.addV("flightpath").property(id,"44").property("shortcode",  "EL").property("name",  "Everlook").property("zone",  "Winterspring").property("continent",  "Kalimdor").property("faction",  "Horde").property("discovered", "True").next()
g.addV("flightpath").property(id,"45").property("shortcode",  "BV").property("name",  "Bloodvenom Post").property("zone",  "Felwood").property("continent",  "Kalimdor").property("faction",  "Horde").property("discovered", "True").next()
g.addV("flightpath").property(id,"46").property("shortcode",  "ZO").property("name",  "Zoram'gar Outpost").property("zone",  "Ashenvale").property("continent",  "Kalimdor").property("faction",  "Horde").property("discovered", "True").next()
g.addV("flightpath").property(id,"47").property("shortcode",  "ST").property("name",  "Splintertree Post").property("zone",  "Ashenvale").property("continent",  "Kalimdor").property("faction",  "Horde").property("discovered", "True").next()
g.addV("flightpath").property(id,"48").property("shortcode",  "VM").property("name",  "Valormok").property("zone",  "Azshara").property("continent",  "Kalimdor").property("faction",  "Horde").property("discovered", "True").next()
g.addV("flightpath").property(id,"49").property("shortcode",  "OG").property("name",  "Orgrimmar").property("zone",  "Durotar").property("continent",  "Kalimdor").property("faction",  "Horde").property("discovered", "True").next()
g.addV("flightpath").property(id,"50").property("shortcode",  "SR").property("name",  "Sun Rock Retreat").property("zone",  "Stonetalon Mountains").property("continent",  "Kalimdor").property("faction",  "Horde").property("discovered", "True").next()
g.addV("flightpath").property(id,"51").property("shortcode",  "SP").property("name",  "Shadowprey Village").property("zone",  "Desolace").property("continent",  "Kalimdor").property("faction",  "Horde").property("discovered", "True").next()
g.addV("flightpath").property(id,"52").property("shortcode",  "TB").property("name",  "Thunder Bluff").property("zone",  "Mulgore").property("continent",  "Kalimdor").property("faction",  "Horde").property("discovered", "True").next()
g.addV("flightpath").property(id,"53").property("shortcode",  "XR").property("name",  "The Crossroads").property("zone",  "The Barrens").property("continent",  "Kalimdor").property("faction",  "Horde").property("discovered", "True").next()
g.addV("flightpath").property(id,"54").property("shortcode",  "RA").property("name",  "Ratchet").property("zone",  "The Barrens").property("continent",  "Kalimdor").property("faction",  "Horde").property("discovered", "True").next()
g.addV("flightpath").property(id,"55").property("shortcode",  "CT").property("name",  "Camp Taurajo").property("zone",  "The Barrens").property("continent",  "Kalimdor").property("faction",  "Horde").property("discovered", "True").next()
g.addV("flightpath").property(id,"56").property("shortcode",  "BW").property("name",  "Brackenwall Village").property("zone",  "Dustwallow Marsh").property("continent",  "Kalimdor").property("faction",  "Horde").property("discovered", "True").next()
g.addV("flightpath").property(id,"57").property("shortcode",  "CM").property("name",  "Camp Mojache").property("zone",  "Feralas").property("continent",  "Kalimdor").property("faction",  "Horde").property("discovered", "True").next()
g.addV("flightpath").property(id,"58").property("shortcode",  "FW").property("name",  "Freewind Post").property("zone",  "Thousand Needles").property("continent",  "Kalimdor").property("faction",  "Horde").property("discovered", "True").next()
g.addV("flightpath").property(id,"59").property("shortcode",  "MR").property("name",  "Marshall's Refuge").property("zone",  "Un'goro Crater").property("continent",  "Kalimdor").property("faction",  "Horde").property("discovered", "True")


1,v[59]


### Create gremlin queries to load edge relationship data
(copy and paste the output from the above cell)

In [143]:
# print all edge add queries
for edge in edges[1:]:
    edge_add_query = 'g.V(\"'+edge[1]+'\").addE(\"'+edge[3]+'\").to(g.V(\"'+edge[2]+'\")).property("faction", \"'+edge[5]+'\").property("discovered", \"'+edge[6]+'\").next()'
    print(edge_add_query)
    #vertex_adds.append('g.addV('+location[7]+').property(id,'+location[0]+').property(shortcode, '+location[1]+').property(name, '+location[2]+').property(zone, '+location[3]+').property(continent, '+location[4]+').property(faction, '+location[5]+').property(discovered, '+location[6]+')')
    

g.V("1").addE("flightpath").to(g.V("2")).property("faction", "Alliance").property("discovered", "discovered").next()
g.V("1").addE("flightpath").to(g.V("3")).property("faction", "Alliance").property("discovered", "discovered").next()
g.V("1").addE("flightpath").to(g.V("7")).property("faction", "Alliance").property("discovered", "discovered").next()
g.V("2").addE("flightpath").to(g.V("3")).property("faction", "Alliance").property("discovered", "discovered").next()
g.V("2").addE("flightpath").to(g.V("4")).property("faction", "Alliance").property("discovered", "discovered").next()
g.V("2").addE("flightpath").to(g.V("7")).property("faction", "Alliance").property("discovered", "discovered").next()
g.V("3").addE("flightpath").to(g.V("4")).property("faction", "Alliance").property("discovered", "discovered").next()
g.V("3").addE("flightpath").to(g.V("5")).property("faction", "Alliance").property("discovered", "discovered").next()
g.V("4").addE("flightpath").to(g.V("5")).property("faction", "Al

In [144]:
%%gremlin
g.V("1").addE("flightpath").to(g.V("2")).property("faction", "Alliance").property("discovered", "discovered").next()
g.V("1").addE("flightpath").to(g.V("3")).property("faction", "Alliance").property("discovered", "discovered").next()
g.V("1").addE("flightpath").to(g.V("7")).property("faction", "Alliance").property("discovered", "discovered").next()
g.V("2").addE("flightpath").to(g.V("3")).property("faction", "Alliance").property("discovered", "discovered").next()
g.V("2").addE("flightpath").to(g.V("4")).property("faction", "Alliance").property("discovered", "discovered").next()
g.V("2").addE("flightpath").to(g.V("7")).property("faction", "Alliance").property("discovered", "discovered").next()
g.V("3").addE("flightpath").to(g.V("4")).property("faction", "Alliance").property("discovered", "discovered").next()
g.V("3").addE("flightpath").to(g.V("5")).property("faction", "Alliance").property("discovered", "discovered").next()
g.V("4").addE("flightpath").to(g.V("5")).property("faction", "Alliance").property("discovered", "discovered").next()
g.V("4").addE("flightpath").to(g.V("6")).property("faction", "Alliance").property("discovered", "discovered").next()
g.V("4").addE("flightpath").to(g.V("7")).property("faction", "Alliance").property("discovered", "discovered").next()
g.V("5").addE("flightpath").to(g.V("6")).property("faction", "Alliance").property("discovered", "discovered").next()
g.V("5").addE("flightpath").to(g.V("8")).property("faction", "Alliance").property("discovered", "discovered").next()
g.V("6").addE("flightpath").to(g.V("8")).property("faction", "Alliance").property("discovered", "discovered").next()
g.V("7").addE("flightpath").to(g.V("8")).property("faction", "Alliance").property("discovered", "discovered").next()
g.V("7").addE("flightpath").to(g.V("9")).property("faction", "Alliance").property("discovered", "discovered").next()
g.V("7").addE("flightpath").to(g.V("4")).property("faction", "Alliance").property("discovered", "discovered").next()
g.V("9").addE("flightpath").to(g.V("10")).property("faction", "Alliance").property("discovered", "discovered").next()
g.V("10").addE("flightpath").to(g.V("11")).property("faction", "Alliance").property("discovered", "discovered").next()
g.V("10").addE("flightpath").to(g.V("15")).property("faction", "Alliance").property("discovered", "discovered").next()
g.V("11").addE("flightpath").to(g.V("12")).property("faction", "Alliance").property("discovered", "discovered").next()
g.V("11").addE("flightpath").to(g.V("15")).property("faction", "Alliance").property("discovered", "discovered").next()
g.V("11").addE("flightpath").to(g.V("14")).property("faction", "Alliance").property("discovered", "discovered").next()
g.V("11").addE("flightpath").to(g.V("13")).property("faction", "Alliance").property("discovered", "discovered").next()
g.V("11").addE("flightpath").to(g.V("16")).property("faction", "Alliance").property("discovered", "discovered").next()
g.V("12").addE("flightpath").to(g.V("14")).property("faction", "Alliance").property("discovered", "discovered").next()
g.V("12").addE("flightpath").to(g.V("13")).property("faction", "Alliance").property("discovered", "discovered").next()
g.V("13").addE("flightpath").to(g.V("14")).property("faction", "Alliance").property("discovered", "discovered").next()
g.V("13").addE("flightpath").to(g.V("16")).property("faction", "Alliance").property("discovered", "discovered").next()
g.V("14").addE("flightpath").to(g.V("16")).property("faction", "Alliance").property("discovered", "discovered").next()
g.V("14").addE("flightpath").to(g.V("15")).property("faction", "Alliance").property("discovered", "discovered").next()
g.V("17").addE("flightpath").to(g.V("18")).property("faction", "Horde").property("discovered", "discovered").next()
g.V("17").addE("flightpath").to(g.V("1")).property("faction", "Horde").property("discovered", "discovered").next()
g.V("17").addE("flightpath").to(g.V("20")).property("faction", "Horde").property("discovered", "discovered").next()
g.V("17").addE("flightpath").to(g.V("21")).property("faction", "Horde").property("discovered", "discovered").next()
g.V("17").addE("flightpath").to(g.V("19")).property("faction", "Horde").property("discovered", "discovered").next()
g.V("17").addE("flightpath").to(g.V("22")).property("faction", "Horde").property("discovered", "discovered").next()
g.V("18").addE("flightpath").to(g.V("19")).property("faction", "Horde").property("discovered", "discovered").next()
g.V("18").addE("flightpath").to(g.V("20")).property("faction", "Horde").property("discovered", "discovered").next()
g.V("18").addE("flightpath").to(g.V("21")).property("faction", "Horde").property("discovered", "discovered").next()
g.V("1").addE("flightpath").to(g.V("20")).property("faction", "Horde").property("discovered", "discovered").next()
g.V("20").addE("flightpath").to(g.V("21")).property("faction", "Horde").property("discovered", "discovered").next()
g.V("21").addE("flightpath").to(g.V("22")).property("faction", "Horde").property("discovered", "discovered").next()
g.V("22").addE("flightpath").to(g.V("9")).property("faction", "Horde").property("discovered", "discovered").next()
g.V("22").addE("flightpath").to(g.V("16")).property("faction", "Horde").property("discovered", "discovered").next()
g.V("22").addE("flightpath").to(g.V("25")).property("faction", "Horde").property("discovered", "discovered").next()
g.V("22").addE("flightpath").to(g.V("23")).property("faction", "Horde").property("discovered", "discovered").next()
g.V("22").addE("flightpath").to(g.V("24")).property("faction", "Horde").property("discovered", "discovered").next()
g.V("9").addE("flightpath").to(g.V("23")).property("faction", "Horde").property("discovered", "discovered").next()
g.V("23").addE("flightpath").to(g.V("24")).property("faction", "Horde").property("discovered", "discovered").next()
g.V("24").addE("flightpath").to(g.V("25")).property("faction", "Horde").property("discovered", "discovered").next()
g.V("24").addE("flightpath").to(g.V("16")).property("faction", "Horde").property("discovered", "discovered").next()
g.V("27").addE("flightpath").to(g.V("29")).property("faction", "Alliance").property("discovered", "discovered").next()
g.V("27").addE("flightpath").to(g.V("28")).property("faction", "Alliance").property("discovered", "discovered").next()
g.V("28").addE("flightpath").to(g.V("29")).property("faction", "Alliance").property("discovered", "discovered").next()
g.V("28").addE("flightpath").to(g.V("31")).property("faction", "Alliance").property("discovered", "discovered").next()
g.V("28").addE("flightpath").to(g.V("30")).property("faction", "Alliance").property("discovered", "discovered").next()
g.V("28").addE("flightpath").to(g.V("35")).property("faction", "Alliance").property("discovered", "discovered").next()
g.V("28").addE("flightpath").to(g.V("39")).property("faction", "Alliance").property("discovered", "discovered").next()
g.V("28").addE("flightpath").to(g.V("33")).property("faction", "Alliance").property("discovered", "discovered").next()
g.V("28").addE("flightpath").to(g.V("34")).property("faction", "Alliance").property("discovered", "discovered").next()
g.V("28").addE("flightpath").to(g.V("51")).property("faction", "Alliance").property("discovered", "discovered").next()
g.V("28").addE("flightpath").to(g.V("37")).property("faction", "Alliance").property("discovered", "discovered").next()
g.V("29").addE("flightpath").to(g.V("30")).property("faction", "Alliance").property("discovered", "discovered").next()
g.V("29").addE("flightpath").to(g.V("31")).property("faction", "Alliance").property("discovered", "discovered").next()
g.V("31").addE("flightpath").to(g.V("35")).property("faction", "Alliance").property("discovered", "discovered").next()
g.V("30").addE("flightpath").to(g.V("35")).property("faction", "Alliance").property("discovered", "discovered").next()
g.V("33").addE("flightpath").to(g.V("51")).property("faction", "Alliance").property("discovered", "discovered").next()
g.V("51").addE("flightpath").to(g.V("34")).property("faction", "Alliance").property("discovered", "discovered").next()
g.V("35").addE("flightpath").to(g.V("36")).property("faction", "Alliance").property("discovered", "discovered").next()
g.V("35").addE("flightpath").to(g.V("39")).property("faction", "Alliance").property("discovered", "discovered").next()
g.V("34").addE("flightpath").to(g.V("39")).property("faction", "Alliance").property("discovered", "discovered").next()
g.V("34").addE("flightpath").to(g.V("37")).property("faction", "Alliance").property("discovered", "discovered").next()
g.V("39").addE("flightpath").to(g.V("38")).property("faction", "Alliance").property("discovered", "discovered").next()
g.V("39").addE("flightpath").to(g.V("42")).property("faction", "Alliance").property("discovered", "discovered").next()
g.V("38").addE("flightpath").to(g.V("42")).property("faction", "Alliance").property("discovered", "discovered").next()
g.V("38").addE("flightpath").to(g.V("37")).property("faction", "Alliance").property("discovered", "discovered").next()
g.V("37").addE("flightpath").to(g.V("40")).property("faction", "Alliance").property("discovered", "discovered").next()
g.V("41").addE("flightpath").to(g.V("40")).property("faction", "Neutral").property("discovered", "discovered").next()
g.V("41").addE("flightpath").to(g.V("42")).property("faction", "Neutral").property("discovered", "discovered").next()
g.V("40").addE("flightpath").to(g.V("42")).property("faction", "Neutral").property("discovered", "discovered").next()
g.V("43").addE("flightpath").to(g.V("45")).property("faction", "Horde").property("discovered", "discovered").next()
g.V("43").addE("flightpath").to(g.V("52")).property("faction", "Horde").property("discovered", "discovered").next()
g.V("43").addE("flightpath").to(g.V("44")).property("faction", "Horde").property("discovered", "discovered").next()
g.V("44").addE("flightpath").to(g.V("45")).property("faction", "Horde").property("discovered", "discovered").next()
g.V("44").addE("flightpath").to(g.V("49")).property("faction", "Horde").property("discovered", "discovered").next()
g.V("45").addE("flightpath").to(g.V("48")).property("faction", "Horde").property("discovered", "discovered").next()
g.V("45").addE("flightpath").to(g.V("49")).property("faction", "Horde").property("discovered", "discovered").next()
g.V("45").addE("flightpath").to(g.V("53")).property("faction", "Horde").property("discovered", "discovered").next()
g.V("48").addE("flightpath").to(g.V("47")).property("faction", "Horde").property("discovered", "discovered").next()
g.V("48").addE("flightpath").to(g.V("53")).property("faction", "Horde").property("discovered", "discovered").next()
g.V("48").addE("flightpath").to(g.V("49")).property("faction", "Horde").property("discovered", "discovered").next()
g.V("46").addE("flightpath").to(g.V("47")).property("faction", "Horde").property("discovered", "discovered").next()
g.V("46").addE("flightpath").to(g.V("53")).property("faction", "Horde").property("discovered", "discovered").next()
g.V("47").addE("flightpath").to(g.V("49")).property("faction", "Horde").property("discovered", "discovered").next()
g.V("47").addE("flightpath").to(g.V("53")).property("faction", "Horde").property("discovered", "discovered").next()
g.V("49").addE("flightpath").to(g.V("52")).property("faction", "Horde").property("discovered", "discovered").next()
g.V("49").addE("flightpath").to(g.V("53")).property("faction", "Horde").property("discovered", "discovered").next()
g.V("49").addE("flightpath").to(g.V("56")).property("faction", "Horde").property("discovered", "discovered").next()
g.V("49").addE("flightpath").to(g.V("42")).property("faction", "Horde").property("discovered", "discovered").next()
g.V("50").addE("flightpath").to(g.V("53")).property("faction", "Horde").property("discovered", "discovered").next()
g.V("50").addE("flightpath").to(g.V("52")).property("faction", "Horde").property("discovered", "discovered").next()
g.V("50").addE("flightpath").to(g.V("51")).property("faction", "Horde").property("discovered", "discovered").next()
g.V("53").addE("flightpath").to(g.V("52")).property("faction", "Horde").property("discovered", "discovered").next()
g.V("53").addE("flightpath").to(g.V("54")).property("faction", "Horde").property("discovered", "discovered").next()
g.V("53").addE("flightpath").to(g.V("56")).property("faction", "Horde").property("discovered", "discovered").next()
g.V("53").addE("flightpath").to(g.V("55")).property("faction", "Horde").property("discovered", "discovered").next()
g.V("53").addE("flightpath").to(g.V("58")).property("faction", "Horde").property("discovered", "discovered").next()
g.V("53").addE("flightpath").to(g.V("57")).property("faction", "Horde").property("discovered", "discovered").next()
g.V("52").addE("flightpath").to(g.V("51")).property("faction", "Horde").property("discovered", "discovered").next()
g.V("52").addE("flightpath").to(g.V("57")).property("faction", "Horde").property("discovered", "discovered").next()
g.V("52").addE("flightpath").to(g.V("58")).property("faction", "Horde").property("discovered", "discovered").next()
g.V("52").addE("flightpath").to(g.V("42")).property("faction", "Horde").property("discovered", "discovered").next()
g.V("52").addE("flightpath").to(g.V("55")).property("faction", "Horde").property("discovered", "discovered").next()
g.V("52").addE("flightpath").to(g.V("56")).property("faction", "Horde").property("discovered", "discovered").next()
g.V("51").addE("flightpath").to(g.V("57")).property("faction", "Horde").property("discovered", "discovered").next()
g.V("55").addE("flightpath").to(g.V("58")).property("faction", "Horde").property("discovered", "discovered").next()
g.V("57").addE("flightpath").to(g.V("58")).property("faction", "Horde").property("discovered", "discovered").next()
g.V("57").addE("flightpath").to(g.V("42")).property("faction", "Horde").property("discovered", "discovered").next()
g.V("57").addE("flightpath").to(g.V("40")).property("faction", "Horde").property("discovered", "discovered").next()
g.V("48").addE("flightpath").to(g.V("42")).property("faction", "Horde").property("discovered", "discovered").next()

1,e[fcb8b9c2-b450-a12d-ea8e-b329205bf884][48-flightpath->42]


## 3. Traversing the graph with Gremlin

### Visualize the vertices in the graph to ensure they were added

In [145]:
%%gremlin
g.V()

1,v[1]
2,v[2]
3,v[3]
4,v[4]
5,v[5]
6,v[6]
7,v[7]
8,v[8]
9,v[9]
10,v[10]
11,v[11]


### Return all edges in the graph

In [146]:
%%gremlin

g.E()

1,e[2eb8b9c2-b37e-94bb-918c-b2f16fe8fea0][5-flightpath->6]
2,e[36b8b9c2-b39e-6869-50c2-6d6ab7158998][13-flightpath->14]
3,e[acb8b9c2-b3bd-70e7-a013-6a05d25e9bba][22-flightpath->9]
4,e[e2b8b9c2-b3e1-d55c-f13f-b02c8f7f51ad][28-flightpath->33]
5,e[1ab8b9c2-b407-c232-3399-0ee0a81eaa0e][38-flightpath->42]
6,e[70b8b9c2-b422-0371-2433-05452d197123][48-flightpath->49]
7,e[dab8b9c2-b43b-82aa-3479-80b495522cdc][53-flightpath->58]
8,e[12b8b9c2-b380-79d6-cbe4-b48d07080786][5-flightpath->8]
9,e[acb8b9c2-b39f-5ac3-164d-76dc7f2c19af][13-flightpath->16]
10,e[64b8b9c2-b3bf-38f9-2039-0088295cd52f][22-flightpath->16]
11,e[0eb8b9c2-b3e2-f484-0c4e-34e208ae80c3][28-flightpath->34]


## Simple access queries

#### Find a flight point by city name

In [147]:
%%gremlin
g.V().has("name", "Ironforge").valueMap();

1,"{'continent': ['Eastern Kingdoms'], 'discovered': ['True'], 'zone': ['Dun Morogh'], 'faction': ['Alliance'], 'name': ['Ironforge'], 'shortcode': ['IF']}"


#### Reference a vertex by id

In [148]:
%%gremlin

g.V('11').valueMap();

1,"{'continent': ['Eastern Kingdoms'], 'discovered': ['True'], 'zone': ['Elwynn Forest'], 'faction': ['Alliance'], 'name': ['Stormwind City'], 'shortcode': ['SW']}"


#### Get all faction-specific flight points for a particular continent
- World of Wacraft Zones may have 1 or more flight masters associated with them, across one or both factions.

In [149]:
%%gremlin
g.V().has("continent","Eastern Kingdoms").and().has("faction","Alliance")

1,v[1]
2,v[2]
3,v[3]
4,v[4]
5,v[5]
6,v[6]
7,v[7]
8,v[8]
9,v[9]
10,v[10]
11,v[11]


#### Get all flight points accessible to a faction
- Note: In World of Warcraft, there are three faction possibilies for a given city; `Horde`, `Alliance`, or `Neutral`.

In [150]:
%%gremlin
g.V().where(has("faction","Horde").or().has("faction","Neutral")).valueMap();

1,"{'continent': ['Eastern Kingdoms'], 'discovered': ['True'], 'zone': ['Tirisfal Glades'], 'faction': ['Horde'], 'name': ['The Undercity'], 'shortcode': ['UC']}"
2,"{'continent': ['Eastern Kingdoms'], 'discovered': ['True'], 'zone': ['Silverpine Forest'], 'faction': ['Horde'], 'name': ['The Sepulcher'], 'shortcode': ['SP']}"
3,"{'continent': ['Eastern Kingdoms'], 'discovered': ['True'], 'zone': ['Hillsbrad Foothills'], 'faction': ['Horde'], 'name': ['Tarren Mill'], 'shortcode': ['TM']}"
4,"{'continent': ['Eastern Kingdoms'], 'discovered': ['True'], 'zone': ['The Hinterlands'], 'faction': ['Horde'], 'name': ['Raventusk Village'], 'shortcode': ['RV']}"
5,"{'continent': ['Eastern Kingdoms'], 'discovered': ['True'], 'zone': ['Arathi Highlands'], 'faction': ['Horde'], 'name': ['Hammerfall'], 'shortcode': ['HF']}"
6,"{'continent': ['Eastern Kingdoms'], 'discovered': ['True'], 'zone': ['Badlands'], 'faction': ['Horde'], 'name': ['Kargath'], 'shortcode': ['KG']}"
7,"{'continent': ['Eastern Kingdoms'], 'discovered': ['True'], 'zone': ['Burning Steppes'], 'faction': ['Horde'], 'name': ['Flame Crest'], 'shortcode': ['FC']}"
8,"{'continent': ['Eastern Kingdoms'], 'discovered': ['True'], 'zone': ['Swamp of Sorrows'], 'faction': ['Horde'], 'name': ['Stonard'], 'shortcode': ['ST']}"
9,"{'continent': ['Eastern Kingdoms'], 'discovered': ['True'], 'zone': ['Stranglethorn Vale'], 'faction': ['Horde'], 'name': [""Grom'gol""], 'shortcode': ['GG']}"
10,"{'continent': ['Eastern Kingdoms'], 'discovered': ['True'], 'zone': ['Silvermoon City'], 'faction': ['Horde'], 'name': [""Quel'danas""], 'shortcode': ['QD']}"
11,"{'continent': ['Kalimdor'], 'discovered': ['True'], 'zone': ['Moonglade'], 'faction': ['Horde'], 'name': ['Nighthaven'], 'shortcode': ['MG']}"


#### Delete a vertex
- There could be a scenario where we want to see what it means to have a city completely removed from our dataset, and dropping the entry makes the most sense.

In [ ]:
%%gremlin

g.V().has('name', 'Booty Bay').drop()

#### Remove eligibility, but not delete
- We can simulate removal or designate lack of eligibility by flipping the 'discovered' feature flag to `False`, and using discovered/prereqs as a filterable field.

9. **Run a traversal**

In [50]:
%%gremlin

g.V().hasLabel('flightpath')

1,v[1]
2,v[2]
3,v[3]
4,v[4]
5,v[5]
6,v[6]
7,v[7]
8,v[8]
9,v[9]
10,v[10]
11,v[11]


## 4. Complex Queries

**Find all surrounding towns with a flightpath into or out of Ironforge**

In [189]:
%%gremlin

g.V().has('name', 'Ironforge').both('flightpath').valueMap()

1,"{'continent': ['Eastern Kingdoms'], 'discovered': ['True'], 'zone': ['Hillsbrad Foothills'], 'faction': ['Alliance'], 'name': ['Southshore'], 'shortcode': ['SS']}"
2,"{'continent': ['Eastern Kingdoms'], 'discovered': ['True'], 'zone': ['Loch Modan'], 'faction': ['Alliance'], 'name': ['Thelsamar'], 'shortcode': ['TH']}"
3,"{'continent': ['Eastern Kingdoms'], 'discovered': ['True'], 'zone': ['Searing Gorge'], 'faction': ['Alliance'], 'name': ['Thorium Point'], 'shortcode': ['TP']}"
4,"{'continent': ['Eastern Kingdoms'], 'discovered': ['True'], 'zone': ['Eastern Plaguelands'], 'faction': ['Alliance'], 'name': [""Light's Hope Chapel""], 'shortcode': ['LH']}"
5,"{'continent': ['Eastern Kingdoms'], 'discovered': ['True'], 'zone': ['Western Plaguelands'], 'faction': ['Alliance'], 'name': ['Chillwind Camp'], 'shortcode': ['CW']}"
6,"{'continent': ['Eastern Kingdoms'], 'discovered': ['True'], 'zone': ['Hillsbrad Foothills'], 'faction': ['Alliance'], 'name': ['Southshore'], 'shortcode': ['SS']}"


#### List all (inbound and outbound) edges for each vertex

In [203]:
%%gremlin

g.V().group().by().by(bothE().count())

1,"{v[44]: 3, v[45]: 5, v[46]: 2, v[47]: 4, v[48]: 5, v[49]: 8, v[50]: 3, v[51]: 6, v[52]: 10, v[53]: 12, v[10]: 3, v[54]: 1, v[11]: 6, v[55]: 3, v[12]: 3, v[56]: 3, v[13]: 4, v[57]: 6, v[14]: 5, v[58]: 4, v[15]: 3, v[59]: 0, v[16]: 5, v[17]: 6, v[18]: 4, v[19]: 2, v[1]: 5, v[2]: 4, v[3]: 4, v[4]: 6, v[5]: 4, v[6]: 3, v[7]: 6, v[8]: 3, v[9]: 4, v[20]: 4, v[21]: 4, v[22]: 7, v[23]: 3, v[24]: 4, v[25]: 2, v[26]: 0, v[27]: 2, v[28]: 10, v[29]: 4, v[30]: 3, v[31]: 3, v[32]: 0, v[33]: 2, v[34]: 4, v[35]: 5, v[36]: 1, v[37]: 4, v[38]: 3, v[39]: 5, v[40]: 4, v[41]: 2, v[42]: 8, v[43]: 3}"


In [208]:
%%gremlin
g.V().groupCount().by(label).toList()

1,"{v[44]: 1, v[45]: 1, v[46]: 1, v[47]: 1, v[48]: 1, v[49]: 1, v[50]: 1, v[51]: 1, v[52]: 1, v[53]: 1, v[10]: 1, v[54]: 1, v[11]: 1, v[55]: 1, v[12]: 1, v[56]: 1, v[13]: 1, v[57]: 1, v[14]: 1, v[58]: 1, v[15]: 1, v[59]: 1, v[16]: 1, v[17]: 1, v[18]: 1, v[19]: 1, v[1]: 1, v[2]: 1, v[3]: 1, v[4]: 1, v[5]: 1, v[6]: 1, v[7]: 1, v[8]: 1, v[9]: 1, v[20]: 1, v[21]: 1, v[22]: 1, v[23]: 1, v[24]: 1, v[25]: 1, v[26]: 1, v[27]: 1, v[28]: 1, v[29]: 1, v[30]: 1, v[31]: 1, v[32]: 1, v[33]: 1, v[34]: 1, v[35]: 1, v[36]: 1, v[37]: 1, v[38]: 1, v[39]: 1, v[40]: 1, v[41]: 1, v[42]: 1, v[43]: 1}"


In [205]:
%%gremlin
g.E().groupCount().by(T.label).toList()

1,{'flightpath': 121}


### Going forward:

- What is the trip with the shortest number of stops between A and B?
- Remove a vertex or edge. How does this effect total travel time along the flight leg?

- Gremlin common recipes: http://tinkerpop.apache.org/docs/current/recipes/#_recipes
- Gremlin common traversal recipes: http://tinkerpop.apache.org/docs/current/recipes/#_traversal_recipes

### Connecting with ipython-gremlin
Visualization:
- What does it look like to graph our vertices and edges?
    - Let Me Graph That for You: https://aws.amazon.com/blogs/database/let-me-graph-that-for-you-part-1-air-routes/
    - iPython Gremlin: https://ipython-gremlin.readthedocs.io/en/latest/
    
 - Graphing bar chart of longest direct routes
 - 